In [1]:
import numpy as np
from scipy.io import loadmat
from sklearn.neighbors import NearestNeighbors

In [2]:
# Load all of the saved files
train_feat = np.loadtxt('train_feat.gzip', dtype = 'float')
gallery_feat = np.loadtxt('gallery_feat.gzip', dtype = 'float')
query_feat = np.loadtxt('query_feat.gzip', dtype = 'float')
val_feat = np.loadtxt('val_feat.gzip', dtype = 'float')
train_noval_feat = np.loadtxt('train_noval_feat.gzip', dtype = 'float')
train_labels = np.loadtxt('train_labels.gzip', dtype = 'uint16')
gallery_labels = np.loadtxt('gallery_labels.gzip', dtype = 'uint16')
query_labels = np.loadtxt('query_labels.gzip', dtype = 'uint16')
val_labels = np.loadtxt('val_labels.gzip', dtype = 'uint16')
train_noval_labels = np.loadtxt('train_noval_labels.gzip', dtype = 'uint16')
#train_filelist = np.loadtxt('train_filelist.gzip', dtype = 'uint16')
#gallery_filelist = np.loadtxt('gallery_filelist.gzip', dtype = 'uint16')
#query_filelist = np.loadtxt('query_filelist.gzip', dtype = 'uint16')
#val_filelist = np.loadtxt('val_filelist.gzip', dtype = 'uint16')
#train_noval_filelist = np.loadtxt('train_noval_filelist.gzip', dtype = 'uint16')
train_camId = np.loadtxt('train_camId.gzip', dtype = 'uint16') # uint8
gallery_camId = np.loadtxt('gallery_camId.gzip', dtype = 'uint16')
query_camId = np.loadtxt('query_camId.gzip', dtype = 'uint16')
val_camId = np.loadtxt('val_camId.gzip', dtype = 'uint16')
train_noval_camId = np.loadtxt('train_noval_camId.gzip', dtype = 'uint16')

In [3]:
# Function that takes as input an index that refers to a query image
# and returns the gallery without the unnecessary pictures in it
# Remove the pictures from the gallery that have the same label and are taken by the same camera as out QUERY[index]
# picture in the query array
def get_removed_gallery (index):
    selected_indices = np.logical_not(np.logical_and(gallery_camId == query_camId[index], gallery_labels == query_labels[index]))
    return gallery_feat[selected_indices, :], gallery_labels[selected_indices]

In [4]:
# k nearest neighbours implementation
k = 10
top_k_accuracies_rank = np.zeros((query_labels.shape[0],k))
top_k_accuracies_classical = np.zeros((query_labels.shape[0],k))
top_k_accuracies_recall = np.zeros((query_labels.shape[0],k))

for i in range (0, query_labels.shape[0], 1):
    gallery_feat_removed, gallery_labels_removed = get_removed_gallery (i)
    nbrs = NearestNeighbors(n_neighbors=k).fit(gallery_feat_removed)
    distances, indices = nbrs.kneighbors(query_feat[i:i+1, :])
    print(gallery_labels_removed[indices], query_labels[i])
    is_same_label = (query_labels[i] == gallery_labels_removed[indices])
    
    for j in range (0, k, 1):
        top_k_accuracies_rank[i,j] = np.sum(is_same_label[:(j+1)]) != 0
        top_k_accuracies_classical[i,j]=np.sum(is_same_label[:(j+1)])/(j+1)
        top_k_accuracies_recall[i,j] = np.sum(is_same_label[:(j+1)])/np.sum(gallery_labels_removed==query_labels[i])
print (top_k_accuracies_rank[:, 0].mean())
print (top_k_accuracies_classical[:, 0].mean())
print (top_k_accuracies_recall[:, 0].mean())

[[1368  739 1426   51 1368   14   14 1450   51 1368]] 3
[[ 517  235  235 1426  725   86   51 1426  172    3]] 3
[[  6   6   6   6 678  77  77 678 678 648]] 6
[[  6   6   6   6 678 678 678 678 678  72]] 6
[[ 22   7 382  22   7 303   7 854   7 441]] 7
[[319 319   7 319   7   7   7 319  22  22]] 7
[[390 738  11 738  11  55  11 625 211  42]] 11
[[738  55 721  11 721 721 401 721 738 763]] 11
[[ 14  14   3  51  14  51  51 182 650  51]] 14
[[ 14  14  14  14 739 291  51 182 182 739]] 14
[[ 17  17  17  17  32 715  32  32  68  68]] 17
[[ 17  17  17  17 715  32 715  32 115 715]] 17
[[ 17  17  17  17  22  17  17   7 715  70]] 22
[[ 22  22  22 303 319 303 319 303 780 319]] 22
[[723 723  28 723  28  28 723 143 723 562]] 28
[[ 28  28  28  28 562 562 143 562 143 562]] 28
[[ 30  30  30  30 585 585 585 585 822 404]] 30
[[ 30  30 404  30  30 404 404 650 404 650]] 30
[[174 174  32 174 174 505 674  32  32 174]] 32
[[32 32 22 11 17 17 32 17 17 22]] 32
[[ 33  33 378 378 378  33 378  70  70  70]] 33
[[ 378  3

[[172 172 172 133 133 235 609 280 190 165]] 172
[[647 647 647 147 674 306 296 258 147 247]] 174
[[340 575 575 652 174 174 321 296 652 640]] 174
[[419 335 335 335 175 335 190 175 175 419]] 175
[[175 190 190 419 175 190 429 429 190 469]] 175
[[ 176  176  176  176 1042  471  471  471 1042  505]] 176
[[176 505 505 426 426 505 426 505 426 471]] 176
[[ 180  180  180  180  396 1044  396 1044 1044  396]] 180
[[ 180  396  180  180  180  396  396  396 1044 1044]] 180
[[182 182 542 542 542 650 205 586 255 650]] 182
[[182 182 182  51 221 426 426  51  14 670]] 182
[[184 184 550 184 550 284 550 550 730 550]] 184
[[184 184 184 184 284 550 550 550 550 284]] 184
[[ 366  366  186  186  175  249  469  127 1389  366]] 186
[[366 366 366 366 740 740 740 723 186 675]] 186
[[ 187  187  187  187  730  730  259 1307  202  730]] 187
[[ 187  187  187 1307  202  259  259  397 1307  259]] 187
[[190 175 609 419 190 609 190 175 175 175]] 190
[[190 190 609 190 635 609 609 249 249 190]] 190
[[193 193 193 150 193  30 15

[[547 337 337 547 337 547 337 372 547 547]] 337
[[337 337 337 337 849 849 849 849 694 547]] 337
[[338 338 338 338 798 674 826 674 798 798]] 338
[[ 247  247  674  247  826  826  674  674  929 1276]] 338
[[331 340 237 331 340 331 303 230 331 340]] 340
[[303 303 331 303 331 340 796 340 340 303]] 340
[[652 652 652 343 343 652 584 616 616 442]] 343
[[ 343  300  822 1445 1445  343 1445  652 1436  300]] 343
[[ 322  322  322  322   68  345  616  409 1022  345]] 345
[[  72   72  345   72  345  345 1022   72 1022  804]] 345
[[ 346  346  346  346  381 1312  699  381  452  749]] 346
[[ 346  346 1312  381  381 1096 1463  452  238 1463]] 346
[[ 485  349  349 1443  216  349  349  548  485  232]] 349
[[ 862  505  148  148  304  557  304  216 1376  349]] 349
[[351 519 219 351 592 219 519 487 351 487]] 351
[[351 351 351 400 351 519 400 400 519 722]] 351
[[357 357 357 357 599 599 599 599 212 212]] 357
[[357 357 357 357 599 599 569 212 578 212]] 357
[[214 398 398 479 398 214 427 398 145 136]] 358
[[ 834  

[[849 849 326 337 804 453 849 849 322 337]] 499
[[504 504 504 578 373 504  54 373 578 578]] 504
[[566 360 129 504 424 460 557  54 424 361]] 504
[[ 463 1376  463  769  537  463  826  769  463 1376]] 505
[[271 504 271 176 560 504 176  28 460 140]] 505
[[ 375  506  195  375 1375  212  195  506  375  212]] 506
[[506 435 506 506 195 506 435 435 375  34]] 506
[[171 523 171 171 672 112 479 145 145 171]] 509
[[523 523 523 523 473 520 473 465 509 465]] 509
[[517 518 517 532 532 517 532 517 532 532]] 517
[[517 532 532 532 517 532 517 532 517 518]] 517
[[267 518 767 559 267 767 187 517 149 518]] 518
[[518 518 496 496 152 496 187 267 518 635]] 518
[[487 519 351 351 592 487 351 351 519 351]] 519
[[ 722  401 1372  459  459  400  459  722  722  367]] 519
[[249 556 419 419 556 419 249 556 419 120]] 520
[[509 523 419 523 419 523 120 509 509 485]] 520
[[220 220 763 220 220 763 316 763 763 763]] 521
[[523 119 119 165 165 374 536 536 675 675]] 521
[[ 522  522 1407  522  522 1407  319  128  237  241]] 522


[[691 691 292 292 292 744 304 304 744 744]] 691
[[691 304 333 666 666 666 691 666 691  81]] 691
[[335 335 586 692 692 692 692 335 190 310]] 692
[[ 390  692  390  692  390 1389  390  652  419  390]] 692
[[372 320 320 372 372 320 547 372 418 418]] 694
[[694 320 694 694 694 320 372 372 320 320]] 694
[[ 187  187  259  187  259 1307 1307  187  187  187]] 695
[[402 402 402 695 695 696 402 708 402 696]] 695
[[696 696 696 696 784 784 195 784 784 708]] 696
[[696 696 696 696 784 784 784 784 695 695]] 696
[[697 697 310 745 440 705 745 745 697 705]] 697
[[697 586 586 586 586 586 586 291 697 291]] 697
[[390 390 343 652 343 699 823 652 652 390]] 699
[[699 390 346 346 699 699 390 699 390 390]] 699
[[703 703 703 597 250 250 250 597 597 597]] 703
[[703 703 703 703 250 597 250 250 250 250]] 703
[[ 704  704  756  704 1259  704  802 1259  616  756]] 704
[[704 704 564 564 564 704 616 745 704 661]] 704
[[ 705  419  705  544  159  705  544 1181  269  705]] 705
[[586 705 705 651 310 586 705  35 310 705]] 705


[[ 838  838 1432 1440 1392  838  317 1347 1432 1443]] 838
[[ 839 1387 1387  839 1387  839  839   47   47   47]] 839
[[ 839 1450 1450 1387  879 1387 1387  879 1387  839]] 839
[[ 840  840  840  840 1300  772  870 1459  332  593]] 840
[[ 840  840  840 1459   91   91  115   91  870  408]] 840
[[ 739 1406  269  739  249 1216 1406  269 1406 1369]] 841
[[1369  120  120 1285  548  267  120  646  646 1369]] 841
[[ 843  843  843  843  569  149  336  491  569 1213]] 843
[[ 843  843   96 1225  149 1213   96  938 1307 1213]] 843
[[ 892  902 1376  902  902  892  899  883 1331  880]] 846
[[ 862 1376 1376  846 1376 1042  862 1376  862  846]] 846
[[ 849  849  849  849 1022 1354 1062 1062 1022 1062]] 849
[[ 849 1022  804 1062 1022 1350 1062 1062 1062  849]] 849
[[ 849  998  223 1132  872 1354 1132  849  907  849]] 851
[[ 879  756  851 1336 1330 1360 1360  879 1336 1330]] 851
[[ 886  899  877  929  869 1030  886  877  886  892]] 853
[[ 886  886  886  853  902  886 1064 1407 1064  853]] 853
[[ 854  873  8

[[1270 1270  975 1028 1170 1061 1270 1032  902 1145]] 967
[[1257 1155 1127  902  965 1037 1037 1127 1155  967]] 967
[[1179 1179 1179 1179  968 1179  968  909  933 1179]] 968
[[ 989  968  932  989  933  933  932 1075  989  932]] 968
[[1162 1162  979 1162  946  980 1271  946 1162 1241]] 969
[[1389 1389 1406  979  969 1007  969  979  925 1007]] 969
[[1122 1063 1063 1068  999 1121  971 1063 1063  988]] 970
[[1064  971 1148  970 1148 1407 1064 1148 1068 1026]] 970
[[ 922  971 1026  758 1068 1148  758  796  971  971]] 971
[[1026 1026 1026  922  922 1121 1068 1128  922  970]] 971
[[1170 1170 1170 1019  975  967  134 1019  893 1266]] 975
[[1114 1114 1114 1375  746 1032  967  762  762 1125]] 975
[[1162  969 1162  969  980 1007  979  969  926  969]] 979
[[ 933  932  932  932  932  834 1075  932 1069 1075]] 979
[[ 965 1077 1273 1195 1273 1273  970 1273 1002 1195]] 980
[[ 926  858  926  858  858 1054 1077  926  862  860]] 980
[[ 982  982  982 1242 1205 1257  965 1155 1112 1257]] 982
[[ 989 1099 10

[[1068 1111 1111 1026  922 1111  922 1068  145 1026]] 1122
[[1122 1122 1375 1375 1375 1122 1375  938 1265  967]] 1122
[[1259  998 1259  980  998 1220 1011 1037 1259  925]] 1123
[[1063 1063  980 1221 1259 1037  998 1009 1011 1063]] 1123
[[1065 1138 1138 1138 1138 1125  924 1065 1125  924]] 1125
[[1138 1138 1125 1138 1138 1125 1138 1125 1138 1125]] 1125
[[ 983 1127 1205  983 1127  983 1205 1205 1257  983]] 1127
[[1127 1276 1276 1276  930 1224 1273 1147  929  930]] 1127
[[ 956  956 1372  956 1128 1128 1372  934  956 1128]] 1128
[[1128  956  956  956  956  956  956 1372  956 1372]] 1128
[[1266 1263 1131 1131 1131 1170 1019 1266 1133 1133]] 1131
[[1131 1131 1266 1131 1263 1263 1019 1233 1266 1019]] 1131
[[1132 1132 1132 1132  851  851 1230 1047  902 1047]] 1132
[[1132 1132 1132  851 1047 1273  597  223 1047 1273]] 1132
[[1133 1133 1134 1266 1266 1131 1133 1128 1131 1131]] 1133
[[1133  338 1140 1128 1133 1134 1168 1140  338  338]] 1133
[[1134 1133 1140 1133 1133 1140 1133 1140 1140 1140]] 11

[[1270 1270 1188 1032 1242 1188 1078 1032 1242 1270]] 1270
[[1216 1162 1112 1162 1273 1112  950 1216 1162 1162]] 1271
[[1097 1097 1153 1075 1097 1017 1097 1017 1017 1017]] 1271
[[1273 1159 1002 1002 1159 1436 1155 1159 1276 1155]] 1273
[[1273 1273 1159 1159 1107 1107 1114 1107  873  319]] 1273
[[ 930 1273 1276 1276 1199 1276 1190  886 1276  886]] 1276
[[1276 1147  929 1190  930 1127 1131 1127 1190  929]] 1276
[[ 887 1305  979  822 1443  822  822   86 1326 1347]] 1285
[[1389  860 1234 1234 1095  509  860 1452 1397 1223]] 1285
[[1286  265  660  265  570  265  265  570 1286  592]] 1286
[[ 784 1286  784  784  784 1286  265  597  597  784]] 1286
[[1293 1293 1293 1293  802  802 1300  802  802  802]] 1293
[[1293 1293  802  802 1300  802  802 1293  802  802]] 1293
[[1359 1326 1359 1359 1359 1326 1331 1422  740  469]] 1298
[[1298  882 1305 1347 1298  882 1359 1359 1359 1359]] 1298
[[1300  320 1342  320  320 1293 1293 1300 1354 1342]] 1300
[[1293 1293  682  645  645  645 1293  600  682  682]] 13

[[1397 1397 1426 1397 1450  650 1426 1426 1368  242]] 1450
[[1451  303 1451 1451  303 1451  319  319  319  319]] 1451
[[ 319  319  319 1451  319  319  319  319  319  281]] 1451
[[1075 1075 1075 1075 1234 1004 1234 1095  739 1004]] 1452
[[841 159 556 419 242 166 442 440 440 159]] 1452
[[1422 1457  186 1457 1443  186 1271 1457  812  812]] 1457
[[ 834 1457 1457 1457  834  834  979  834  933 1443]] 1457
[[1347 1347 1445 1426  836 1342 1445 1426  836 1359]] 1459
[[1445 1445  882 1384 1384  816 1445 1397  816 1445]] 1459
[[1418 1418 1418 1461 1418 1418 1461  584 1461  584]] 1461
[[1418 1418 1461 1418 1418 1461 1461 1461 1418 1293]] 1461
[[1463 1463 1463  905 1463  884 1435  884  884  905]] 1463
[[1463 1463  884  884 1463  884  346  884  346  905]] 1463
0.7492857142857143
2.1471428571428572
0.5636904761904762


In [8]:
print (is_same_label, is_same_label.shape)

[[False False False False  True False False False False  True]] (1, 10)


In [5]:
a, b = get_removed_gallery(0)
print (a, a.shape, b, b.shape)

[[0.76888168 0.25586957 0.60124129 ... 0.18115909 0.43165341 0.09931619]
 [1.39672875 0.77182716 0.70401114 ... 0.01811628 0.08462342 0.12913167]
 [1.30696166 0.22219124 0.62375975 ... 0.02954952 0.37348923 0.05303315]
 ...
 [0.01961832 0.17617127 0.54575312 ... 0.33026204 0.93925601 0.56220758]
 [0.05719076 0.60635394 0.58300596 ... 0.40994745 1.37061644 0.55975795]
 [0.07245111 0.51656747 0.59082419 ... 0.60488421 1.05610693 0.72939718]] (5325, 2048) [   3    3    3 ... 1463 1463 1463] (5325,)


In [6]:
# OLD k nearest neighbours implementation
k = 10
print (query_labels.shape)
top_10_accuracies = np.zeros((query_labels.shape[0]))
top_5_accuracies = np.zeros((query_labels.shape[0]))
top_1_accuracies = np.zeros((query_labels.shape[0]))
accuracies = np.zeros((query_labels.shape[0]))
for i in range (0, query_labels.shape[0], 1):
    gallery_feat_removed, gallery_labels_removed = get_removed_gallery (i)
    nbrs = NearestNeighbors(n_neighbors=k).fit(gallery_feat_removed)
    distances, indices = nbrs.kneighbors(query_feat[i:i+1, :])
    print(gallery_labels_removed[indices], query_labels[i])
    is_same_label = (query_labels[i] == gallery_labels_removed[indices])
    
    
    top_10_accuracies[i] = np.sum(is_same_label) != 0
    top_5_accuracies[i] = np.sum(is_same_label[:5]) != 0
    top_1_accuracies[i] = np.sum(is_same_label[:1]) != 0
    accuracies[i]=np.sum(is_same_label)/k
    print (accuracies[i])
    print (top_1_accuracies[i])
    print (top_5_accuracies[i])
    print (top_10_accuracies[i])
    
print (accuracies.mean())
print (top_1_accuracies.mean())
print (top_5_accuracies.mean())
print (top_10_accuracies.mean())

(1400,)
[[1368  739 1426   51 1368   14   14 1450   51 1368]] 3
0.0
0.0
0.0
0.0
[[ 517  235  235 1426  725   86   51 1426  172    3]] 3
0.1
1.0
1.0
1.0
[[  6   6   6   6 678  77  77 678 678 648]] 6
0.4
1.0
1.0
1.0
[[  6   6   6   6 678 678 678 678 678  72]] 6
0.4
1.0
1.0
1.0
[[ 22   7 382  22   7 303   7 854   7 441]] 7
0.4
1.0
1.0
1.0
[[319 319   7 319   7   7   7 319  22  22]] 7
0.4
1.0
1.0
1.0
[[390 738  11 738  11  55  11 625 211  42]] 11
0.3
1.0
1.0
1.0
[[738  55 721  11 721 721 401 721 738 763]] 11
0.1
1.0
1.0
1.0
[[ 14  14   3  51  14  51  51 182 650  51]] 14
0.3
1.0
1.0
1.0
[[ 14  14  14  14 739 291  51 182 182 739]] 14
0.4
1.0
1.0
1.0
[[ 17  17  17  17  32 715  32  32  68  68]] 17
0.4
1.0
1.0
1.0
[[ 17  17  17  17 715  32 715  32 115 715]] 17
0.4
1.0
1.0
1.0
[[ 17  17  17  17  22  17  17   7 715  70]] 22
0.1
1.0
1.0
1.0
[[ 22  22  22 303 319 303 319 303 780 319]] 22
0.3
1.0
1.0
1.0
[[723 723  28 723  28  28 723 143 723 562]] 28
0.3
1.0
1.0
1.0
[[ 28  28  28  28 562 562 143 562

[[143 143 143 143 562  28 723 732 723 732]] 143
0.4
1.0
1.0
1.0
[[143 143 562 143 562 143 723 732 723 562]] 143
0.4
1.0
1.0
1.0
[[ 194  194  410  145 1440  584  594  410  194 1440]] 145
0.1
1.0
1.0
1.0
[[171 638 145 145 465 518 530 171 465 145]] 145
0.3
1.0
1.0
1.0
[[673 146 146 146 649 146 673 673 673 673]] 146
0.4
1.0
1.0
1.0
[[673 673 673 673 649 649 661 649 673 649]] 146
0.0
0.0
0.0
0.0
[[505 163 317 647 349 672 652 157 157 652]] 147
0.0
0.0
0.0
0.0
[[129 544 157 129 133 163 151 544 147 151]] 147
0.1
1.0
1.0
1.0
[[544 147 148 643 148 148 643 792 148 792]] 148
0.4
1.0
1.0
1.0
[[228 349 643 643 129 129 129 157 643 147]] 148
0.0
0.0
0.0
0.0
[[149 695 149 112 136 720 695 402 398 112]] 149
0.2
1.0
1.0
1.0
[[149 149 149 149 824 770 720 112 112 770]] 149
0.4
1.0
1.0
1.0
[[150 150 556 419 150 150 556 419 556 585]] 150
0.4
1.0
1.0
1.0
[[150 150 150 150 419 585 193 419 585 585]] 150
0.4
1.0
1.0
1.0
[[599 558  74 151 213 599 427 599 134 558]] 151
0.1
1.0
1.0
1.0
[[ 151  427  335  278  151  13

[[249 186 269 186 249 186 331 186 429  98]] 249
0.2
1.0
1.0
1.0
[[331 237 166 331 237 254 249 726 166 237]] 249
0.1
1.0
1.0
1.0
[[250 250 236 250 250 236 157 645 703 703]] 250
0.4
1.0
1.0
1.0
[[250 250 250 703 250 157 236 236 236 236]] 250
0.4
1.0
1.0
1.0
[[635 239 239 166 542 235 726 522 669 522]] 254
0.0
0.0
0.0
0.0
[[166 239 254  98 239 239 254 635 239 726]] 254
0.2
1.0
1.0
1.0
[[255 255 182 213 182 213 182 255 255  69]] 255
0.4
1.0
1.0
1.0
[[182 182 255 182 182 182 182 255 255 182]] 255
0.3
1.0
1.0
1.0
[[ 646 1353  675  296 1353  280 1353  646  646  748]] 258
0.0
0.0
0.0
0.0
[[147 647 647 304 147 647 292 147 647 211]] 258
0.0
0.0
0.0
0.0
[[1225 1225 1225 1225  452  259  259 1225  259 1463]] 259
0.3
1.0
1.0
1.0
[[ 259  259  202 1307  187  695  187 1225 1307  187]] 259
0.2
1.0
1.0
1.0
[[261 261 261 648 648  40 836 816 836 261]] 261
0.4
1.0
1.0
1.0
[[ 261  261  261  261  332 1228  332  265  594  816]] 261
0.4
1.0
1.0
1.0
[[269 300 614 584  33 614 265 665 816 822]] 265
0.1
1.0
1.0
1.0


[[379 379 379  42 379 437 437  74  74  60]] 379
0.4
1.0
1.0
1.0
[[379 379 623 623 379 291 651 310  74 566]] 379
0.3
1.0
1.0
1.0
[[238 238 238 238 238 499 238 238 499 381]] 381
0.1
1.0
1.0
1.0
[[ 381  346  381  381  346  559 1312  381  346   76]] 381
0.4
1.0
1.0
1.0
[[521 367 521 367 521 401 367 367 763 374]] 382
0.0
0.0
0.0
0.0
[[367 367 763 471 521 367 471 471 521 374]] 382
0.0
0.0
0.0
0.0
[[384 384 401 401 367 401 459 384 769 459]] 384
0.3
1.0
1.0
1.0
[[384 384 384 400 401 401 463 401 382 135]] 384
0.3
1.0
1.0
1.0
[[394 378 378 402 388 429 388 394 556 378]] 388
0.2
1.0
1.0
1.0
[[388 479 479 388 378 479 573 112 112 479]] 388
0.2
1.0
1.0
1.0
[[699 652 699 699 699 699 699 699 692 692]] 390
0.0
0.0
0.0
0.0
[[640 640 409 409 640 521 220 220 220 640]] 390
0.0
0.0
0.0
0.0
[[ 33 394 394 394  77  77 408   6  72  33]] 394
0.3
1.0
1.0
1.0
[[378 394 378  33 394 388  33  33 394 397]] 394
0.3
1.0
1.0
1.0
[[180 180 180 180 396 180 180 180 396 180]] 396
0.2
1.0
1.0
1.0
[[396 396 241 180 396 396  41 

KeyboardInterrupt: 

In [ ]:



well_predicted = gallery_labels_removed[indices, :] == np.ones((k))*query_labels[0]
print (well_predicted, well_predicted.shape)
precision_or_recall = np.sum(well_predicted)/k
print (precision_or_recall)

In [ ]:
print (query_feat.dtype, query_feat.max())
print (gallery_feat, gallery_feat.shape)
print (query_feat, query_feat.shape)

In [ ]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors([[0, 1], [0,0]])
print(indices)
print (distances)

In [ ]:
k = 5
nbrs = NearestNeighbors(n_neighbors=k).fit(gallery_feat)
distances, indices = nbrs.kneighbors(query_feat)
print(indices, indices.shape)
print (distances, distances.shape)